# Computer Vision V/S  NLP

### Note:- You have to accept NLP first in your subconcious mind if you have some experience in computer vision.

- I have written above sentence because it happened to me as I was having some experience in computer vision and I don't know how I was facing problems while studying NLP than one of my friend had given me this advice that -" hey, yash you are not accepting nlp try to accept it". after this sentence I don't know how I had worked on NLP for 2 months which was not possible for me before, I'm writing this so that it may help someone like me:)

- Now I think many of the good kernels are already present which will teach you NLP much better than me, the only thing I want to convey through this notebook is how a beginner will approach a NLP problem.

- there are mainly two steps in nlp - Tokenization(converting words to tokens)+Model(RNNs/LSTMs?Transformers)

- The code is self explanatory and only thing I want to convey just read any others well descripted kernel than have some initial approach in your mind and then see this kernel and repeat this definitely you will develop confidence in nlp since this is one of the most basic dataset in NLP.

- After repeating it please upload your notebook as that will encourage to write a documented code and improve your confidence in sharing things in the community:)

- to hindi speakers - 
  "NLP karo aur apna knowledge uplift karo,
  ab begginer mahashay grand master banne ki taiyaari karo".
  
- Please if you find any thing useful in it Don't forgot to upvote:)
   

In [ ]:
#imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing import sequence
from sklearn.metrics import confusion_matrix
import itertools

In [ ]:
#Reading data
df1 = pd.read_csv('/kaggle/input/fake-and-real-news-dataset/True.csv')
df1['label'] = 1
df2 = pd.read_csv('/kaggle/input/fake-and-real-news-dataset/Fake.csv')
df2['label'] = 0
df = pd.concat([df1,df2])

In [ ]:
df.head()

In [ ]:
plt.hist(df['text'].apply(lambda x: len(x.split())))

In [ ]:
#Splitting the dataset
X = df.text
y = df.label
#converting label format so that model understand it
y = to_categorical(y, num_classes = 2)
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size = 0.1)

In [ ]:
"""
These steps are very much important since these steps make your language data into the type of data on which RNN works.
for eg. ["I am a boy","I like kaggle",....] --(tokenization)-->[[1 2 3 4],[1 5 6],....]

"""
vocab = 10000  #maximum no. of words in our dictionary
max_len = 600  #maximum length of sentence (if more than that sentence will be trucated otherwise "0" will be added to complete max len)
tokens = Tokenizer(num_words = vocab)
tokens.fit_on_texts(X_train)
sequences = tokens.texts_to_sequences(X_train)
sequence_matrix = sequence.pad_sequences(sequences, maxlen = max_len)
#for test data
test_sequences = tokens.texts_to_sequences(X_test)
test_sequence_matrix = sequence.pad_sequences(test_sequences, maxlen = max_len)

In [ ]:
"""
This step seems to be very easy but what is going behind this ie. how LSTM works, I have tried demonstrate an idea but you are
free to do comments if find any thing unrelatable:)
 
 
                                      [prob. of real, prob. of fake]
                                                   |
                                          _____________________
                                         |                     |
                                         |_____________________|
                                          |        |        |
                                         ___      ___      ___
                                        |   |    |   |    |   |
                                    ----|   |----|   |----|   |--
                                        |___|    |___|    |___|
                                          |        |        |
                                         ___      ___      ___
                                        |   |    |   |    |   |
                                    ----|   |----|   |----|   |--
                                        |___|    |___|    |___|
                                          |        |        |
                                          I       like     Kaggle

"""
model = Sequential()
model.add(Embedding(vocab,300,input_length = max_len))
model.add(LSTM(64))
model.add(Dense(1000, activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(2,activation = 'sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
#Fit the model along with a validation split
model.fit(sequence_matrix,y_train,validation_split = 0.2,epochs= 5)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.1f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.show()

In [ ]:
%matplotlib inline
labels = ["Real", "Fake"]

# plotting confusion matrix
Y_pred = model.predict(np.array(test_sequence_matrix))
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
Y_true = np.argmax(np.array(y_test),axis = 1) 

# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes = labels,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues)
